In [ ]:
import time
import pyautogui
from selenium import webdriver
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

#Usuario y contraseña
user = '45725395'
password = '$JC90*Kdk#'
#Filas a consultar
beginning = 2
end = 50
#Constantes para CHG-BTS
affectation = "2G - 3G - 4G"
worst_case = "Fuera de servicio 2G, 3G y 4G"
duration = "180:00"
group = "CLARO"
owner = "Raul Zuniga"
job = "JP3367" 
# Constantes para las columnas
COLUMN_SITE = 'A'
COLUMN_OT_UM = 'B'
COLUMN_ID_REF = 'C'
#API sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
KEY = 'key.json'
SPREADSHEET_ID = '1ay_j1TVixDqlUwK1G4s3XkqseoXa8OcXpfyJE7qWHRE'
creds = service_account.Credentials.from_service_account_file(KEY, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

def get_sheet_values(sheet, column, beginning, end):
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=f'BTS!{column}{beginning}:{column}{end}'
    ).execute()
    return result.get('values', [])

def get_spreadsheet_data(sheet, beginning, end):
    site = get_sheet_values(sheet, COLUMN_SITE, beginning, end)
    ot_um = get_sheet_values(sheet, COLUMN_OT_UM, beginning, end)
    id_rf = get_sheet_values(sheet, COLUMN_ID_REF, beginning, end)

    return site, ot_um, id_rf
    
def append_values_to_spreadsheet(sheet, spreadsheet_id, range_, values, sheet_name):
    result = sheet.values().append(
        spreadsheetId=spreadsheet_id,
        range=f'{sheet_name}!{range_}',
        valueInputOption='USER_ENTERED',
        body={'values': values}
    ).execute()
    return result
    
def find_and_click(id):
    time.sleep(0.7)
    element = wait.until(EC.presence_of_element_located((By.ID, id)))
    element.click()

def find_and_send_keys(id,key):
    time.sleep(0.7)
    element = wait.until(EC.presence_of_element_located((By.ID, id)))
    element.send_keys(key)

def login():
    time.sleep(0.2)
    pyautogui.typewrite(user)
    pyautogui.press('tab')
    pyautogui.typewrite(password)
    pyautogui.press('enter')
    time.sleep(0.2)


SITE, OT_UM, ID_RF = get_spreadsheet_data(sheet, beginning, end)
#Inicialización webdriver e ingreso a la página maximo
driver = webdriver.Chrome()
driver.get('http://100.126.20.133/maximo/ui/login')
#Login
login()
wait = WebDriverWait(driver, 20)  # Se define una espera de hasta 20 segundos
#Nuevo cambio
find_and_click('FavoriteApp_CHANGE')
for i in range(len(SITE)):
    project = "Implementación y Adecuación 5G" #Dato necesario para el resumen, se debe agregar en el formulario
    site = SITE[i][0]
    try:
        ot_um = OT_UM[i][0]
    except IndexError:
        ot_um = ""
        pass
    try:
        id_rf = ID_RF[i][0]
    except IndexError:
        id_rf = ""
        pass
    resume = site+"_"+project
    find_and_click('toolactions_INSERT-tbb_image')
    #Obtención de CHG
    time.sleep(1)
    chg = wait.until(EC.presence_of_element_located((By.ID, 'm77a6936f-lb')))
    chg = chg.text
    print(chg)
    #Casilla formato
    find_and_click('m7247dd9b-cb_img')
    #Destino principal
    find_and_send_keys('mb627369d-tb',site)
    find_and_click('mb627369d-img')                  
    time.sleep(1.5)
    #Programa
    find_and_click('m356798d1-tab')
    #Plan de trabajo
    time.sleep(2)
    find_and_send_keys('me1cea54e-tb',job)
    find_and_click('me1cea54e-img')
    #Resumen
    time.sleep(4)
    find_and_click('mc3e99bf4-tb')
    pyautogui.press('delete')
    find_and_send_keys('mc3e99bf4-tb',resume)
    #Fecha de planificación
    if duration=="180:00":
        start = datetime.now().replace(hour=9, minute=0, second=0, microsecond=0)
        format_date_time = "%d/%m/%y %H:%M:%S"
        start_program = start+ timedelta(days=6)
    else:
        start = datetime.now().replace(hour=10, minute=0, second=0, microsecond=0)
        format_date_time = "%d/%m/%y %H:%M:%S"
        start_program = start + timedelta(days=2)
    end = start_program + timedelta(days=14, hours=9)
    end_program = end
    #Inicio programado
    find_and_click('m8b12679c-tb')
    find_and_send_keys('m8b12679c-tb',str(start_program.strftime(format_date_time)))
    #Inicio previsto
    find_and_click('m651c06b0-tb')
    find_and_send_keys('m651c06b0-tb',str(start_program.strftime(format_date_time)))
    #Finalización programada
    find_and_click('m1576f23f-tb')
    find_and_send_keys('m1576f23f-tb',str(end_program.strftime(format_date_time)))
    #Finalización prevista
    find_and_click('mfc15570a-tb')
    find_and_send_keys('mfc15570a-tb',str(end.strftime(format_date_time)))
    #Duración estimada
    find_and_click('m8c7fa385-tb')
    pyautogui.typewrite(duration)
    find_and_click('m8c7fa385-tb')
    pyautogui.press('enter')
    #Publicación a medios masivos
    #Tiempo minutos fuera de servicio
    time.sleep(0.5)
    m, s = map(int, duration.split(":"))
    find_and_click('m3a8b61b-tb')
    find_and_send_keys('m3a8b61b-tb',str(m))
    #Fecha fuera de servicio
    time.sleep(0.5)
    find_and_send_keys('m74af868d-tb',str(start_program.strftime(format_date_time)))
    find_and_click('m9dcc23b8-tb')
    #Fechas restantes
    time.sleep(0.5)
    find_and_click('mdcfda8d9-tb')
    start_program = start_program + timedelta(days=3)
    find_and_send_keys('mdcfda8d9-tb',str(start_program.strftime(format_date_time)))
    start_program = start_program + timedelta(days=1)
    find_and_click('md870d46b-tb')
    find_and_send_keys('md870d46b-tb',str(start_program.strftime(format_date_time)))
    start_program = start_program + timedelta(days=1)
    find_and_click('m4861a193-tb')
    find_and_send_keys('m4861a193-tb',str(start_program.strftime(format_date_time)))
    start_program = start_program + timedelta(days=1)
    find_and_click('m6d0afe4f-tb')
    find_and_send_keys('m6d0afe4f-tb',str(start_program.strftime(format_date_time)))
    #Ventana cambiar
    find_and_click('mbf28cd64-tab')
    #Afectación contemplada
    time.sleep(2)
    find_and_send_keys('m59e302be-tb',affectation)
    #Afectación peor caso
    time.sleep(0.5)
    find_and_send_keys('m139acc5d-tb',worst_case)
    #Nodo
    if ot_um != "":
        find_and_send_keys('m480e224e-tb',ot_um)
    #Cuenta matriz
    if id_rf != "":
        find_and_send_keys('m12b84828-tb',id_rf)
    #Seleccionar propietario
    find_and_click('md86fe08f_ns_menu_OWNER_OPTION_a_tnode')
    find_and_click('m2578bad6-lb2')
    find_and_send_keys('m2578bad6_tfrow_[C:0]_txt-tb',group)
    time.sleep(0.7)
    find_and_send_keys('m2578bad6_tfrow_[C:3]_txt-tb',owner)
    time.sleep(1.2)
    pyautogui.press('enter')
    time.sleep(0.2)
    find_and_click('m2578bad6_tfrow_[C:3]_txt-tb')
    time.sleep(0.2)
    num_tabs = 4
    for _ in range(num_tabs):
        pyautogui.press('tab')
        time.sleep(0.2)
    pyautogui.press('enter')

    time.sleep(2)


    start_program = start + timedelta(days=2)
    values_col_D = [[chg]]
    values_col_E = [[str(start_program.strftime(format_date_time))]]

    result_col_D = append_values_to_spreadsheet(
        sheet, SPREADSHEET_ID, f'D{i+2}', values_col_D, sheet_name='BTS'
    )

    result_col_E = append_values_to_spreadsheet(
        sheet, SPREADSHEET_ID, f'E{i+2}', values_col_E, sheet_name='BTS'
    )
    time.sleep(2)
    
    find_and_click('ROUTEWF_CHPRCLARUT_-tbb_image')
    time.sleep(7)
    find_and_click('ROUTEWF_CHPRCLARUT_-tbb_image')
    find_and_click('m67326ef-rb')
    find_and_click('md875c1f9-pb')
    #find_and_click('toolactions_INSERT-tbb_image')
    time.sleep(1.5)